In [1]:
import os
import time
from typing import List, Dict, Tuple

import cohere
import faiss
import google.generativeai as genai
import numpy as np
import pandas as pd
import optuna
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from rank_bm25 import BM25Okapi

# Load environment variables from a .env file
load_dotenv()

# Configure the Google Generative AI client
try:
    genai.configure(api_key=os.getenv('GEMINI_API_KEY'))
except Exception as e:
    print(f"Error configuring Google Generative AI: {e}")
    # Exit or handle the error as appropriate
    exit()

class ImprovedRAG:
    """
    A RAG (Retrieval-Augmented Generation) system with improved retrieval strategies.
    This class handles document embedding, indexing, and answer generation.
    """
    def __init__(self, embedding_dim: int = 384):
        """
        Initializes the RAG system.

        Args:
            embedding_dim (int): The dimension of the text embeddings.
        """
        self.embedding_dim = embedding_dim
        # Initialize a FAISS index for efficient similarity search
        self.index = faiss.IndexFlatL2(embedding_dim)
        # Load a pre-trained model for creating text embeddings
        self.embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        # Store document metadata
        self.documents: List[Dict] = []
        # Initialize the Cohere client for re-ranking
        self.cohere_client = cohere.Client(os.getenv('COHERE_API_KEY'))
        # BM25 index will be created when documents are added
        self.bm25 = None
        # Store tokenized documents for the BM25 index
        self.tokenized_docs = []
        print(f"RAG system initialized with embedding dimension: {embedding_dim}")

    def embed_text_batch(self, texts: List[str]) -> np.ndarray:
        """
        Generates embeddings for a list of texts in batches.

        Args:
            texts (List[str]): A list of text strings to embed.

        Returns:
            np.ndarray: A numpy array of embeddings.
        """
        embeddings = self.embedding_model.embed_documents(texts)
        return np.array(embeddings, dtype='float32')

    def add_documents(self, documents_df: pd.DataFrame):
        """
        Processes and adds documents to the RAG system using semantic chunking.

        Args:
            documents_df (pd.DataFrame): DataFrame containing the documents to add.
                                         It must have '_id', 'title', and 'text' columns.
        """
        print("Loading local embedding model for semantic chunking...")
        # Use semantic chunking to split documents based on meaning
        splitter = SemanticChunker(
            embeddings=self.embedding_model,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )

        texts_to_embed = []
        chunks_meta = []
        tokenized_chunks = []

        # Iterate over the documents and create chunks
        for _id, title, text in documents_df[['_id', 'title', 'text']].itertuples(index=False, name=None):
            chunks = splitter.split_text(text)
            for chunk in chunks:
                full_text = f"Title: {title}\n\n{chunk}"
                texts_to_embed.append(full_text)
                chunks_meta.append({'original_doc_id': _id, 'title': title, 'text': chunk})
                tokenized_chunks.append((title + " " + chunk).lower().split())

        # Generate embeddings and add them to the FAISS index
        if texts_to_embed:
            embeddings = self.embed_text_batch(texts_to_embed)
            self.index.add(embeddings)

        # Build the BM25 index for keyword-based search
        self.tokenized_docs.extend(tokenized_chunks)
        self.bm25 = BM25Okapi(self.tokenized_docs)
        self.documents.extend(chunks_meta)
        print(f"Added {len(chunks_meta)} chunks. Index size: {self.index.ntotal}")
        print(f"BM25 index built with {len(self.tokenized_docs)} documents")

    def retrieve(self, query: str, k: int = 3) -> List[Tuple[Dict, float]]:
        """
        Retrieves relevant documents for a given query using a hybrid approach.

        Args:
            query (str): The user's query.
            k (int): The final number of documents to return.

        Returns:
            List[Tuple[Dict, float]]: A list of tuples, each containing a document
                                     and its relevance score.
        """
        initial_k = 20  # Number of candidates to fetch initially

        # 1. Vector Search (FAISS)
        query_embedding = self.embed_text_batch([query]).reshape(1, -1)
        distances, indices = self.index.search(query_embedding, initial_k)
        vector_results = {idx: rank + 1 for rank, (idx, _) in enumerate(zip(indices[0], distances[0])) if idx < len(self.documents)}

        # 2. Keyword Search (BM25)
        tokenized_query = query.lower().split()
        bm25_scores = self.bm25.get_scores(tokenized_query)
        top_bm25_indices = np.argsort(bm25_scores)[::-1][:initial_k]
        bm25_results = {idx: rank + 1 for rank, idx in enumerate(top_bm25_indices) if idx < len(self.documents)}

        # 3. Reciprocal Rank Fusion (RRF) to combine results
        rrf_k = 60
        rrf_scores = {}
        all_indices = set(vector_results.keys()) | set(bm25_results.keys())
        for idx in all_indices:
            score = 0
            if idx in vector_results:
                score += 1 / (rrf_k + vector_results[idx])
            if idx in bm25_results:
                score += 1 / (rrf_k + bm25_results[idx])
            rrf_scores[idx] = score

        # Sort by RRF score to get the best candidates
        sorted_indices = sorted(rrf_scores.keys(), key=lambda x: rrf_scores[x], reverse=True)
        top_fused_indices = sorted_indices[:initial_k]

        candidates = [{'doc': self.documents[idx], 'text': f"Title: {self.documents[idx]['title']}\n{self.documents[idx]['text']}"} for idx in top_fused_indices]

        if not candidates:
            return []

        # 4. Re-rank with Cohere for final ordering
        rerank_docs = [c['text'] for c in candidates]
        rerank_results = self.cohere_client.rerank(
            query=query,
            documents=rerank_docs,
            top_n=min(k, len(rerank_docs)),
            model='rerank-english-v3.0'
        )

        final_results = []
        for result in rerank_results.results:
            original_candidate = candidates[result.index]
            final_results.append((original_candidate['doc'], result.relevance_score))

        return final_results

    def generate_answer(self, query: str, retrieved_docs: List[Tuple[Dict, float]]) -> str:
        """
        Generates an answer using the retrieved documents as context.

        Args:
            query (str): The user's query.
            retrieved_docs (List[Tuple[Dict, float]]): Documents retrieved for the query.

        Returns:
            str: The generated answer.
        """
        context = "\n\n".join([f"Title: {doc['title']}\n{doc['text']}" for doc, _ in retrieved_docs])
        prompt = f"""
        Context information is provided below.
        ---------------------
        {context}
        ---------------------
        Given the context information and not prior knowledge, answer the query.
        Query: {query}
        Answer:
        """
        try:
            model = genai.GenerativeModel('gemini-2.5-flash')
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating answer: {e}"


class TunableRAG(ImprovedRAG):
    """
    An extension of the ImprovedRAG system that allows for hyperparameter tuning.
    """
    def __init__(self, embedding_dim: int = 384):
        """
        Initializes the TunableRAG system.
        """
        super().__init__(embedding_dim)

    def add_documents_with_threshold(self, documents_df: pd.DataFrame, breakpoint_threshold_amount: int = 95):
        """
        Adds documents with a configurable semantic chunking threshold.
        """
        self.documents = []
        self.tokenized_docs = []
        self.index = faiss.IndexFlatL2(self.embedding_dim)

        splitter = SemanticChunker(
            embeddings=self.embedding_model,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=breakpoint_threshold_amount
        )

        texts_to_embed = []
        chunks_meta = []
        tokenized_chunks = []

        for _id, title, text in documents_df[['_id', 'title', 'text']].itertuples(index=False, name=None):
            chunks = splitter.split_text(text)
            for chunk in chunks:
                full_text = f"Title: {title}\n\n{chunk}"
                texts_to_embed.append(full_text)
                chunks_meta.append({'original_doc_id': _id, 'title': title, 'text': chunk})
                tokenized_chunks.append((title + " " + chunk).lower().split())

        if texts_to_embed:
            embeddings_array = self.embed_text_batch(texts_to_embed)
            self.index.add(embeddings_array)

        self.tokenized_docs.extend(tokenized_chunks)
        self.bm25 = BM25Okapi(self.tokenized_docs)
        self.documents.extend(chunks_meta)

    def retrieve_with_params(self, query: str, initial_k: int, final_k: int) -> List[Tuple[Dict, float]]:
        """
        Retrieves documents with configurable k parameters for initial retrieval and final ranking.
        """
        # This method is largely the same as the parent's `retrieve` but uses the passed k values.
        query_embedding = self.embed_text_batch([query]).reshape(1, -1)
        distances, indices = self.index.search(query_embedding, initial_k)
        vector_results = {idx: rank + 1 for rank, idx in enumerate(indices[0]) if idx < len(self.documents)}

        tokenized_query = query.lower().split()
        bm25_scores = self.bm25.get_scores(tokenized_query)
        top_bm25_indices = np.argsort(bm25_scores)[::-1][:initial_k]
        bm25_results = {idx: rank + 1 for rank, idx in enumerate(top_bm25_indices)}

        rrf_k = 60
        rrf_scores = {}
        all_indices = set(vector_results.keys()) | set(bm25_results.keys())
        for idx in all_indices:
            score = 0
            if idx in vector_results:
                score += 1 / (rrf_k + vector_results[idx])
            if idx in bm25_results:
                score += 1 / (rrf_k + bm25_results[idx])
            rrf_scores[idx] = score

        sorted_indices = sorted(rrf_scores.keys(), key=lambda x: rrf_scores[x], reverse=True)
        top_fused_indices = sorted_indices[:initial_k]

        candidates = [{'doc': self.documents[idx], 'text': f"Title: {self.documents[idx]['title']}\n{self.documents[idx]['text']}"} for idx in top_fused_indices]

        if not candidates:
            return []

        rerank_docs = [c['text'] for c in candidates]
        rerank_results = self.cohere_client.rerank(
            query=query,
            documents=rerank_docs,
            top_n=min(final_k, len(rerank_docs)),
            model='rerank-english-v3.0'
        )

        final_results = [(candidates[res.index]['doc'], res.relevance_score) for res in rerank_results.results]
        return final_results

    def generate_answer_with_temp(self, query: str, retrieved_docs: List[Tuple[Dict, float]], temperature: float) -> str:
        """
        Generates an answer with a configurable temperature for the generative model.
        """
        context = "\n\n".join([f"Title: {doc['title']}\n{doc['text']}" for doc, _ in retrieved_docs])
        prompt = f"""
        Context information is provided below.
        ---------------------
        {context}
        ---------------------
        Given the context information and not prior knowledge, answer the query.
        Query: {query}
        Answer:
        """
        generation_config = genai.types.GenerationConfig(temperature=temperature)
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(prompt, generation_config=generation_config)
        return response.text

/Users/shenhao/Desktop/RAG_system/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/shenhao/Desktop/RAG_system/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- 1. Load Data ---
try:
    corpus_df = pd.read_csv('./assets/corpus.csv')
    queries_df = pd.read_csv('./assets/queries.csv')
    ground_truth_df = pd.read_csv('./assets/train.csv')
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure corpus.csv, queries.csv, and train.csv are in the './assets/' directory.")
    exit()

# --- 2. Prepare Evaluation DataFrame ---
queries_df.rename(columns={'_id': 'query-id'}, inplace=True)
ground_truth_grouped = ground_truth_df.groupby('query-id')['corpus-id'].apply(list).reset_index()
ground_truth_grouped.rename(columns={'corpus-id': 'ground_truth_doc_ids'}, inplace=True)
eval_df = pd.merge(queries_df, ground_truth_grouped, on='query-id')
print(f"Loaded {len(eval_df)} queries for evaluation.")


# --- 3. Define the Objective Function for Optuna ---
def objective(trial):
    """
    Objective function for Optuna to optimize RAG hyperparameters.
    """
    # Suggest hyperparameters for this trial
    breakpoint_threshold = trial.suggest_int("breakpoint_threshold", 90, 99)
    initial_k = trial.suggest_categorical("initial_k", [20, 25, 30, 35, 40])
    final_k = trial.suggest_int("final_k", 3, 10)
    temperature = trial.suggest_categorical("temperature", [0.1, 0.5, 0.9])

    print(f"\nTrial {trial.number}: Testing params - threshold={breakpoint_threshold}, "
            f"initial_k={initial_k}, final_k={final_k}, temp={temperature}")

    # Initialize RAG system with new parameters
    rag_tunable = TunableRAG()

    # Add documents with the specified chunking threshold
    print("  Adding documents with new chunking...")
    rag_tunable.add_documents_with_threshold(corpus_df, breakpoint_threshold_amount=breakpoint_threshold)

    eval_results = []
    # Use a subset of queries for faster tuning
    eval_subset = eval_df.head(10)

    for _, row in eval_subset.iterrows():
        query_text = row['text']
        retrieved_docs = rag_tunable.retrieve_with_params(query_text, initial_k, final_k)
        generated_answer = rag_tunable.generate_answer_with_temp(query_text, retrieved_docs, temperature)
        contexts = [doc['text'] for doc, _ in retrieved_docs]
        eval_results.append({'question': query_text, 'contexts': contexts, 'answer': generated_answer})
        time.sleep(6)  # To avoid hitting API rate limits

    print("  Evaluating performance...")
    model = genai.GenerativeModel('gemini-2.5-flash')
    faithfulness_scores, relevancy_scores, precision_scores = [], [], []

    for item in eval_results:
        # Simplified evaluation logic from the notebook
        try:
            faithfulness_prompt = f"Question: {item['question']}\nContext: {' '.join(item['contexts'])}\nAnswer: {item['answer']}\n\nRate how faithful the answer is to the context (0-1). Return only a number."
            faithfulness = float(model.generate_content(faithfulness_prompt).text.strip())
            time.sleep(6)  # To avoid hitting API rate limits
            relevancy_prompt = f"Question: {item['question']}\nAnswer: {item['answer']}\n\nRate how relevant the answer is to the question (0-1). Return only a number."
            relevancy = float(model.generate_content(relevancy_prompt).text.strip())
            time.sleep(6)  # To avoid hitting API rate limits
            precision_prompt = f"Question: {item['question']}\nContext: {' '.join(item['contexts'])}\n\nRate how precise the context is for answering the question (0-1). Return only a number."
            precision = float(model.generate_content(precision_prompt).text.strip())
            time.sleep(6)  # To avoid hitting API rate limits
        except (ValueError, AttributeError, IndexError):
            faithfulness, relevancy, precision = 0.5, 0.5, 0.5 # Default on failure

        faithfulness_scores.append(faithfulness)
        relevancy_scores.append(relevancy)
        precision_scores.append(precision)

    avg_faithfulness = np.mean(faithfulness_scores)
    avg_relevancy = np.mean(relevancy_scores)
    avg_precision = np.mean(precision_scores)

    # Weighted score to optimize for
    score = (0.5 * avg_relevancy) + (0.4 * avg_precision) + (0.1 * avg_faithfulness)

    print(f"  Scores - Faithfulness: {avg_faithfulness:.3f}, Relevancy: {avg_relevancy:.3f}, Precision: {avg_precision:.3f}")
    print(f"  Final Weighted Score: {score:.3f}")

    return score

# --- 4. Run the Optuna Study ---
study = optuna.create_study(direction="maximize", study_name="rag_hyperparameter_tuning")
print("\nStarting hyperparameter optimization...")
print("=" * 60)
# n_trials can be increased for a more thorough search
study.optimize(objective, n_trials=20)

# --- 5. Display Results ---
print("\n" + "=" * 60)
print("Optimization Complete!")
print("=" * 60)
print("\n🏆 BEST HYPERPARAMETERS FOUND:")
print("-" * 40)
for param, value in study.best_params.items():
    print(f"  {param}: {value}")
print(f"\n🚀 Best Score Achieved: {study.best_value:.4f}")


[I 2025-08-26 22:57:54,383] A new study created in memory with name: rag_hyperparameter_tuning


Loaded 2590 queries for evaluation.

Starting hyperparameter optimization...

Trial 0: Testing params - threshold=95, initial_k=25, final_k=5, temp=0.9


/var/folders/9n/v_tm1cxd4r7c_3mh9hb1ww340000gn/T/ipykernel_69161/2512835882.py:43: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


RAG system initialized with embedding dimension: 384
  Adding documents with new chunking...
  Evaluating performance...


[I 2025-08-26 23:08:08,364] Trial 0 finished with value: 0.93 and parameters: {'breakpoint_threshold': 95, 'initial_k': 25, 'final_k': 5, 'temperature': 0.9}. Best is trial 0 with value: 0.93.


...

  Scores - Faithfulness: 0.960, Relevancy: 1.000, Precision: 0.880
  Final Weighted Score: 0.948

Trial 18: Testing params - threshold=98, initial_k=30, final_k=9, temp=0.1
RAG system initialized with embedding dimension: 384
  Adding documents with new chunking...
  Evaluating performance...


[I 2025-08-27 01:54:13,746] Trial 18 finished with value: 0.912 and parameters: {'breakpoint_threshold': 98, 'initial_k': 30, 'final_k': 9, 'temperature': 0.1}. Best is trial 15 with value: 0.962.


  Scores - Faithfulness: 0.960, Relevancy: 1.000, Precision: 0.790
  Final Weighted Score: 0.912

Trial 19: Testing params - threshold=92, initial_k=20, final_k=8, temp=0.1
RAG system initialized with embedding dimension: 384
  Adding documents with new chunking...
  Evaluating performance...


[I 2025-08-27 02:03:34,659] Trial 19 finished with value: 0.906 and parameters: {'breakpoint_threshold': 92, 'initial_k': 20, 'final_k': 8, 'temperature': 0.1}. Best is trial 15 with value: 0.962.


  Scores - Faithfulness: 0.990, Relevancy: 0.990, Precision: 0.780
  Final Weighted Score: 0.906

Optimization Complete!

🏆 BEST HYPERPARAMETERS FOUND:
----------------------------------------
  breakpoint_threshold: 97
  initial_k: 20
  final_k: 10
  temperature: 0.1

🚀 Best Score Achieved: 0.9620


In [8]:
! pip install plotly
! pip install --upgrade nbformat

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import joblib
import optuna.visualization as vis

# Load the study from the file
study = joblib.load("completed_study.pkl")
print("Study loaded successfully!")

/Users/shenhao/Desktop/RAG_system/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Study loaded successfully!


In [2]:
import optuna.visualization as vis
import matplotlib.pyplot as plt

# --- 6. Visualize the Tuning Results ---

# Plot 1: Optimization History
# Shows how the best score improved over the trials.
print("Plotting Optimization History...")
fig_history = vis.plot_optimization_history(study)
fig_history.show()

# Plot 2: Parameter Importances
# Shows which hyperparameters had the biggest impact on the final score.
print("\nPlotting Parameter Importances...")
fig_importance = vis.plot_param_importances(study)
fig_importance.show()

# Plot 3: Slice Plot
# Shows how each hyperparameter individually affected the score.
print("\nPlotting Slice Plot...")
fig_slice = vis.plot_slice(study)
fig_slice.show()

# Plot 4: Parallel Coordinate Plot
# Helps visualize which combinations of parameters lead to the best scores.
print("\nPlotting Parallel Coordinate Plot...")
fig_parallel = vis.plot_parallel_coordinate(study)
fig_parallel.show()

Plotting Optimization History...



Plotting Parameter Importances...



Plotting Slice Plot...



Plotting Parallel Coordinate Plot...


In [11]:
import joblib

# Save the entire study object to a file
joblib.dump(study, "completed_study.pkl") 
print("Study saved successfully!")

Study saved successfully!
